# Health Canada DPD Download & Extraction Creation

## Imports

In [28]:
import urllib.request
import pandas as pd 
from pandas import DataFrame, Series
pd.set_option('display.max_columns', None)
import os
#import openpyxl as xl # pip install 
import zipfile as zf
import numpy as np
from datetime import date, time, datetime
import chardet # Appears to be an encoding problem with the therapeutic_class file.
# encoding='ISO-8859-1' ??

### Get Zip File of All HC DPD Files

In [55]:
%%time
# Health Canada Read me File
# https://www.canada.ca/en/health-canada/services/drugs-health-products/drug-products/drug-product-database/read-file-drug-product-database-data-extract.html
# Source URL for Health Canada DPD extracts, allfiles.zip
base_url = 'http://www.hc-sc.gc.ca/dhp-mps/alt_formats/zip/prodpharma/databasdon/'
archive_list = ('allfiles.zip', 'allfiles_ia.zip', 'allfiles_ap.zip', 'allfiles_dr.zip')

for arc in archive_list:
    url = base_url + arc
    urllib.request.urlretrieve(url, arc)

archive = zf.ZipFile('allfiles.zip')
# #archive.namelist()        
archive_ia = zf.ZipFile('allfiles_ia.zip')
archive_ap = zf.ZipFile('allfiles_ap.zip')
archive_dr = zf.ZipFile('allfiles_dr.zip')


Wall time: 7.99 s


### Extract Required Files

In [56]:
# Marketed (active) Products
drug_data = archive.extract('drug.txt', '.')
form_data = archive.extract('form.txt', '.')
ingred_data = archive.extract('ingred.txt', '.')
package_data = archive.extract('package.txt', '.')
route_data = archive.extract('route.txt', '.')
therapeutic_data = archive.extract('ther.txt', '.')

# Cancelled (inactivated) Products
drug_data_ia = archive_ia.extract('drug_ia.txt', '.')
form_data_ia = archive_ia.extract('form_ia.txt', '.')
ingred_data_ia = archive_ia.extract('ingred_ia.txt', '.')
package_data_ia = archive_ia.extract('package_ia.txt', '.')
route_data_ia = archive_ia.extract('route_ia.txt', '.')
therapeutic_data_ia = archive_ia.extract('ther_ia.txt', '.')
inactives = archive_ia.extract('inactive.txt', '.')
# The file 'inactive.txt' which identifies products that are no longer approved or marketed (active) 
# in Canada is still available. The following information fields are included in the file:
#  •DRUG_CODE NUMBER (8)
#  •DRUG_IDENTIFICATION_NUMBER VARCHAR2(8)
#  •BRAND_NAME VARCHAR2(200)
#  •HISTORY_DATE DATE

# Approved Products
drug_data_ap = archive_ap.extract('drug_ap.txt', '.')
form_data_ap = archive_ap.extract('form_ap.txt', '.')
ingred_data_ap = archive_ap.extract('ingred_ap.txt', '.')
package_data_ap = archive_ap.extract('package_ap.txt', '.')
route_data_ap = archive_ap.extract('route_ap.txt', '.')
therapeutic_data_ap = archive_ap.extract('ther_ap.txt', '.')

# Dormant Products
drug_data_dr = archive_dr.extract('drug_dr.txt', '.')
form_data_dr = archive_dr.extract('form_dr.txt', '.')
ingred_data_dr = archive_dr.extract('ingred_dr.txt', '.')
package_data_dr = archive_dr.extract('package_dr.txt', '.')
route_data_dr = archive_dr.extract('route_dr.txt', '.')
therapeutic_data_dr = archive_dr.extract('ther_dr.txt', '.')

In [57]:
# Keep in mind the default import "data type" of the DRUG_IDENTIFICATION_NUMBER column will be int64
# this will result in the data loosing any preceeding zeros.  Using dtype=dict of types, you can
# specify the column(s) you want and their data types, as a key:value pair.
drug = pd.read_csv(drug_data, header=None, dtype={'DRUG_IDENTIFICATION_NUMBER': 'object'}, names=['DRUG_CODE','PRODUCT_CATEGORIZATION', 'CLASS', 'DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'DESCRIPTOR','PEDIATRIC_FLAG','ACCESSION_NUMBER','NUMBER_OF_AIS','LAST_UPDATE_DATE','AI_GROUP_NO','CLASS_F','BRAND_NAME_F','DESCRIPTOR_F'])
drug_ia = pd.read_csv(drug_data_ia, header=None, dtype={'DRUG_IDENTIFICATION_NUMBER': 'object'}, names=['DRUG_CODE','PRODUCT_CATEGORIZATION', 'CLASS', 'DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'DESCRIPTOR','PEDIATRIC_FLAG','ACCESSION_NUMBER','NUMBER_OF_AIS','LAST_UPDATE_DATE','AI_GROUP_NO','CLASS_F','BRAND_NAME_F','DESCRIPTOR_F'])
drug_ap = pd.read_csv(drug_data_ap, header=None, dtype={'DRUG_IDENTIFICATION_NUMBER': 'object'}, names=['DRUG_CODE','PRODUCT_CATEGORIZATION', 'CLASS', 'DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'DESCRIPTOR','PEDIATRIC_FLAG','ACCESSION_NUMBER','NUMBER_OF_AIS','LAST_UPDATE_DATE','AI_GROUP_NO','CLASS_F','BRAND_NAME_F','DESCRIPTOR_F'])
drug_dr = pd.read_csv(drug_data_dr, header=None, dtype={'DRUG_IDENTIFICATION_NUMBER': 'object'}, names=['DRUG_CODE','PRODUCT_CATEGORIZATION', 'CLASS', 'DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'DESCRIPTOR','PEDIATRIC_FLAG','ACCESSION_NUMBER','NUMBER_OF_AIS','LAST_UPDATE_DATE','AI_GROUP_NO','CLASS_F','BRAND_NAME_F','DESCRIPTOR_F'])

## Load Data Files into DataFrames

### ACTIVES

In [58]:
form = pd.read_csv(form_data, header=None, names=['DRUG_CODE','PHARM_FORM_CODE','PHARMACEUTICAL_FORM','PHARMACEUTICAL_FORM_F'])
route = pd.read_csv(route_data, header=None, names=['DRUG_CODE', 'ROUTE_OF_ADMINISTATION_CODE', 'ROUTE_OF_ADMINISTRATION', 'ROUTE_OF_ADMINISTRATION_F' ])
ingred = pd.read_csv(ingred_data, header=None, names=['DRUG_CODE', 'ACTIVE_INGREDIENT_CODE', 'INGREDIENT', 'INGREDIENT_SUPPLIED_IND', 'STRENGTH', 'STRENGTH_UNIT', 'STRENGTH_TYPE', 'DOSAGE_VALUE', 'BASE', 'DOSAGE_UNIT', 'NOTES', 'INGREDIENT_F', 'STRENGTH_UNIT_F', 'STRENGTH_TYPE_F', 'DOSAGE_UNIT_F']) 
thera = pd.read_csv(therapeutic_data, header=None, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
pkg = pd.read_csv(package_data, header=None, names=['DRUG_CODE', 'UPC', 'PACKAGE_SIZE_UNIT', 'PACAKGE_TYPE', 'PACKAGE_SIZE', 'PRODUCT_INFORMATION','PACKAGE_SIZE_UNIT_F', 'PACKAGE_TYPE_F'])

### INACTIVES

In [59]:
form_ia = pd.read_csv(form_data_ia, header=None, names=['DRUG_CODE','PHARM_FORM_CODE','PHARMACEUTICAL_FORM','PHARMACEUTICAL_FORM_F'])
route_ia = pd.read_csv(route_data_ia, header=None, names=['DRUG_CODE', 'ROUTE_OF_ADMINISTATION_CODE', 'ROUTE_OF_ADMINISTRATION', 'ROUTE_OF_ADMINISTRATION_F' ])
ingred_ia = pd.read_csv(ingred_data_ia, header=None, names=['DRUG_CODE', 'ACTIVE_INGREDIENT_CODE', 'INGREDIENT', 'INGREDIENT_SUPPLIED_IND', 'STRENGTH', 'STRENGTH_UNIT', 'STRENGTH_TYPE', 'DOSAGE_VALUE', 'BASE', 'DOSAGE_UNIT', 'NOTES', 'INGREDIENT_F', 'STRENGTH_UNIT_F', 'STRENGTH_TYPE_F', 'DOSAGE_UNIT_F']) 
thera_ia = pd.read_csv(therapeutic_data_ia, header=None, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
pkg_ia = pd.read_csv(package_data_ia, header=None, names=['DRUG_CODE', 'UPC', 'PACKAGE_SIZE_UNIT', 'PACAKGE_TYPE', 'PACKAGE_SIZE', 'PRODUCT_INFORMATION','PACKAGE_SIZE_UNIT_F', 'PACKAGE_TYPE_F'])
inactive_ia = pd.read_csv(inactives, header=None, names=['DRUG_CODE_NUMBER','DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'HISTORY_DATE'])

### APPROVED

In [60]:
form_ap = pd.read_csv(form_data_ap, header=None, names=['DRUG_CODE','PHARM_FORM_CODE','PHARMACEUTICAL_FORM','PHARMACEUTICAL_FORM_F'])
route_ap = pd.read_csv(route_data_ap, header=None, names=['DRUG_CODE', 'ROUTE_OF_ADMINISTATION_CODE', 'ROUTE_OF_ADMINISTRATION', 'ROUTE_OF_ADMINISTRATION_F' ])
ingred_ap = pd.read_csv(ingred_data_ap, header=None, names=['DRUG_CODE', 'ACTIVE_INGREDIENT_CODE', 'INGREDIENT', 'INGREDIENT_SUPPLIED_IND', 'STRENGTH', 'STRENGTH_UNIT', 'STRENGTH_TYPE', 'DOSAGE_VALUE', 'BASE', 'DOSAGE_UNIT', 'NOTES', 'INGREDIENT_F', 'STRENGTH_UNIT_F', 'STRENGTH_TYPE_F', 'DOSAGE_UNIT_F']) 
thera_ap = pd.read_csv(therapeutic_data_ap, header=None, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
pkg_ap = pd.read_csv(package_data_ap, header=None, names=['DRUG_CODE', 'UPC', 'PACKAGE_SIZE_UNIT', 'PACAKGE_TYPE', 'PACKAGE_SIZE', 'PRODUCT_INFORMATION','PACKAGE_SIZE_UNIT_F', 'PACKAGE_TYPE_F'])

### DORMANT

In [61]:
form_dr = pd.read_csv(form_data_dr, header=None, names=['DRUG_CODE','PHARM_FORM_CODE','PHARMACEUTICAL_FORM','PHARMACEUTICAL_FORM_F'])
route_dr = pd.read_csv(route_data_dr, header=None, names=['DRUG_CODE', 'ROUTE_OF_ADMINISTATION_CODE', 'ROUTE_OF_ADMINISTRATION', 'ROUTE_OF_ADMINISTRATION_F' ])
ingred_dr = pd.read_csv(ingred_data_dr, header=None, names=['DRUG_CODE', 'ACTIVE_INGREDIENT_CODE', 'INGREDIENT', 'INGREDIENT_SUPPLIED_IND', 'STRENGTH', 'STRENGTH_UNIT', 'STRENGTH_TYPE', 'DOSAGE_VALUE', 'BASE', 'DOSAGE_UNIT', 'NOTES', 'INGREDIENT_F', 'STRENGTH_UNIT_F', 'STRENGTH_TYPE_F', 'DOSAGE_UNIT_F']) 
thera_dr = pd.read_csv(therapeutic_data_dr, header=None, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
pkg_dr = pd.read_csv(package_data_dr, header=None, names=['DRUG_CODE', 'UPC', 'PACKAGE_SIZE_UNIT', 'PACAKGE_TYPE', 'PACKAGE_SIZE', 'PRODUCT_INFORMATION','PACKAGE_SIZE_UNIT_F', 'PACKAGE_TYPE_F'])

In [63]:
# Merge Drug + Form DataFrames
drug_form = pd.merge(drug[drug['CLASS']=='Human'], form, on='DRUG_CODE', how='inner', indicator=True)
drug_form_ia = pd.merge(drug_ia[drug_ia['CLASS']=='Human'], form_ia, on='DRUG_CODE', how='inner', indicator=True)
# drug_form_ap = pd.merge(drug_ap[drug_ia['CLASS']=='Human'], form_ap, on='DRUG_CODE', how='inner', indicator=True)
# drug_form_dr = pd.merge(drug_dr[drug_dr['CLASS']=='Human'], form_dr, on='DRUG_CODE', how='inner', indicator=True)

In [64]:
drug_form_ap = pd.merge(drug_ap[drug_ia['CLASS']=='Human'], form_ap, on='DRUG_CODE', how='inner', indicator=True)
drug_form_dr = pd.merge(drug_dr[drug_dr['CLASS']=='Human'], form_dr, on='DRUG_CODE', how='inner', indicator=True)

C:\WinPython-32bit-3.6.3.0Qt5\python-3.6.3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [10]:
# Merge First Merged DataFrames with Ingredient DataFrame
drug_form_ingred = pd.merge(drug_form, ingred, on='DRUG_CODE', how='inner')
drug_form_ingred_ia = pd.merge(drug_form_ia, ingred_ia, on='DRUG_CODE', how='inner')
drug_form_ingred_ap = pd.merge(drug_form_ap, ingred_ap, on='DRUG_CODE', how='inner')
drug_form_ingred_dr = pd.merge(drug_form_dr, ingred_dr, on='DRUG_CODE', how='inner')

In [13]:
#import chardet # Appears to be an encoding problem with the therapeutic_class file.
# encoding='ISO-8859-1' ??

In [65]:
# Determine Encoding
with open('ther.txt', 'rb') as f:
    result = chardet.detect(f.read())
with open('ther_ia.txt', 'rb') as f:
    result_ia = chardet.detect(f.read())
with open('ther_ap.txt', 'rb') as f:
    result_ap = chardet.detect(f.read())
with open('ther_dr.txt', 'rb') as f:
    result_dr = chardet.detect(f.read())
# Pass Encoding into read_csv
therap = pd.read_csv(therapeutic_data, encoding=result['encoding'], header=None, dtype={'TC_ATC_NUMBER': 'object', 'TC_ATC': 'object', 'TC_AHFS_NUMBER': 'object', 'TC_AHFS': 'object'}, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
therap_ia = pd.read_csv(therapeutic_data_ia, encoding=result_ia['encoding'], header=None, dtype={'TC_ATC_NUMBER': 'object', 'TC_ATC': 'object', 'TC_AHFS_NUMBER': 'object', 'TC_AHFS': 'object'}, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
therap_ap = pd.read_csv(therapeutic_data_ap, encoding=result_ap['encoding'], header=None, dtype={'TC_ATC_NUMBER': 'object', 'TC_ATC': 'object', 'TC_AHFS_NUMBER': 'object', 'TC_AHFS': 'object'}, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])
therap_dr = pd.read_csv(therapeutic_data_dr, encoding=result_dr['encoding'], header=None, dtype={'TC_ATC_NUMBER': 'object', 'TC_ATC': 'object', 'TC_AHFS_NUMBER': 'object', 'TC_AHFS': 'object'}, names=['DRUG_CODE', 'TC_ATC_NUMBER', 'TC_ATC', 'TC_AHFS_NUMBER', 'TC_AHFS', 'TC_ATC_F','TC_AHFS_F'])

In [66]:
drug_form_ingred_route = pd.merge(drug_form_ingred, route, on='DRUG_CODE', how='inner')
drug_form_ingred_route_ia = pd.merge(drug_form_ingred_ia, route_ia, on='DRUG_CODE', how='inner')
drug_form_ingred_route_ap = pd.merge(drug_form_ingred_ap, route_ap, on='DRUG_CODE', how='inner')
drug_form_ingred_route_dr = pd.merge(drug_form_ingred_dr, route_dr, on='DRUG_CODE', how='inner')

In [72]:
drug_form_ingred_route_pkg = pd.merge(drug_form_ingred_route, pkg, on='DRUG_CODE', how='inner')
drug_form_ingred_route_pkg_ia = pd.merge(drug_form_ingred_route, pkg_ia, on='DRUG_CODE', how='inner')
drug_form_ingred_route_pkg_ap = pd.merge(drug_form_ingred_route_ap, pkg_ap, on='DRUG_CODE', how='inner')
drug_form_ingred_route_pkg_dr = pd.merge(drug_form_ingred_route_dr, pkg_dr, on='DRUG_CODE', how='inner')

In [73]:
drug_form_ingred_route_pkg.sort_values(by=['INGREDIENT'])
drug_form_ingred_route_pkg_ia.sort_values(by=['INGREDIENT'])
drug_form_ingred_route_pkg_ap.sort_values(by=['INGREDIENT'])
drug_form_ingred_route_pkg_dr.sort_values(by=['INGREDIENT'])

,DRUG_CODE,PRODUCT_CATEGORIZATION,CLASS,DRUG_IDENTIFICATION_NUMBER,BRAND_NAME,DESCRIPTOR,PEDIATRIC_FLAG,ACCESSION_NUMBER,NUMBER_OF_AIS,LAST_UPDATE_DATE,AI_GROUP_NO,CLASS_F,BRAND_NAME_F,DESCRIPTOR_F,PHARM_FORM_CODE,PHARMACEUTICAL_FORM,PHARMACEUTICAL_FORM_F,_merge,ACTIVE_INGREDIENT_CODE,INGREDIENT,INGREDIENT_SUPPLIED_IND,STRENGTH,STRENGTH_UNIT,STRENGTH_TYPE,DOSAGE_VALUE,BASE,DOSAGE_UNIT,NOTES,INGREDIENT_F,STRENGTH_UNIT_F,STRENGTH_TYPE_F,DOSAGE_UNIT_F,ROUTE_OF_ADMINISTATION_CODE,ROUTE_OF_ADMINISTRATION,ROUTE_OF_ADMINISTRATION_F,UPC,PACKAGE_SIZE_UNIT,PACAKGE_TYPE,PACKAGE_SIZE,PRODUCT_INFORMATION,PACKAGE_SIZE_UNIT_F,PACKAGE_TYPE_F
2171,87320,NaN,Human,02387603,ACETAMINOPHEN,NaN,N,NaN,1,09-AUG-2017,102009002,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,6/20,NaN,NaN
1662,75452,"LS - ANALG. (ACETAMINOPHEN, SALICYLATES, CODEINE)",Human,02269120,EXTRA STRENGTH ACETAMINOPHEN 500MG EASY TO SWA...,NaN,N,NaN,1,14-SEP-2017,102009002,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,100,NaN,NaN
1942,82565,NaN,Human,02340089,PMS RELIEF EXTRA STRENGTH,NaN,N,NaN,3,26-JUL-2017,316117002,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,16/32,NaN,NaN
591,21989,NaN,Human,01935275,ACETAMINOPHEN PEDIATRIC DROPS 80 MG/ML USP,NaN,N,E1842,1,14-SEP-2017,102009003,Humain,NaN,NaN,12,DROPS,Gouttes,both,270,ACETAMINOPHEN,I,80.000,MG,NaN,NaN,N,ML,HISTORIC FREEFORM TEXT: ACETAMINOPHEN,Acétaminophène,MG,NaN,ML,56,ORAL,Orale,NaN,NaN,NaN,NaN,24ML,NaN,NaN
1122,62483,NaN,Human,02239031,RELAXOPHEN CAPLETS,NaN,N,NaN,2,26-JUL-2017,212017001,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,325.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,18/40,NaN,NaN
1947,82884,NaN,Human,02343371,JAMP ACETAMINOPHEN 500 MG F.C. TABLETS,NaN,N,NaN,1,14-SEP-2017,102009002,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,100/150/200/1000,NaN,NaN
2329,91078,NaN,Human,02426188,EXTRA STRENGTH COLD MEDICATION DAY TIME RELIEF,NaN,N,NaN,3,13-SEP-2017,351264005,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,10/20,NaN,NaN
2220,88167,NaN,Human,02396637,SINUS MEDICATION,NaN,N,NaN,3,09-AUG-2017,351580003,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,325.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,6/20,NaN,NaN
1404,70284,NaN,Human,02246141,SINUS RELIEF EXTRA STRENGTH TABLET,NaN,N,NaN,2,14-SEP-2017,216113003,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,500.000,MG,NaN,NaN,Y,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,24,NaN,NaN
2217,88166,NaN,Human,02396629,ALLERGY SINUS MEDICATION,NaN,N,NaN,3,09-AUG-2017,316099002,Humain,NaN,NaN,85,TABLET,Comprimé,both,270,ACETAMINOPHEN,I,325.000,MG,NaN,NaN,N,NaN,NaN,Acétaminophène,MG,NaN,NaN,56,ORAL,Orale,NaN,NaN,NaN,NaN,4/20,NaN,NaN


### Export to SQLite3 Database

In [74]:
import sqlite3
conn = sqlite3.connect('hc_dpd.db')
drug_form_ingred_route_pkg.to_sql(name='actives', con=conn, if_exists='replace')
drug_form_ingred_route_pkg_ia.to_sql(name='inactives', con=conn, if_exists='replace')
inactive_ia.to_sql(name='inactive_list', con=conn, if_exists='replace')
drug_form_ingred_route_pkg_ap.to_sql(name='approved', con=conn, if_exists='replace')
drug_form_ingred_route_pkg_dr.to_sql(name='dormant', con=conn, if_exists='replace')

### Remove Duplicate Rows

In [21]:
# hcdpd_dataset3 = hcdpd_dataset2.drop_duplicates()

In [22]:
# final_dataset = hcdpd_dataset2[['COMPKEY', 'DRUG_CODE', 'DRUG_IDENTIFICATION_NUMBER', 'BRAND_NAME', 'HighALERT', 'TallMan', 'INGREDIENT', 'STRENGTH', 'STRENGTH_UNIT', 'STRENGTH_TYPE', 'PHARMACEUTICAL_FORM', 'DOSAGE_VALUE', 'BASE', 'DOSAGE_UNIT', 'ROUTE_OF_ADMINISTRATION', 'TC_ATC', 'TC_ATC_NUMBER', 'TC_AHFS_NUMBER', 'TC_AHFS']]

## Write DataFrame DataSet to Excel

In [23]:
# Write DataFrame to Excel
##%%time
currentDate = date.today()
createDate = str(currentDate.month) + str(currentDate.year)

# if os.path.exists('HC_DPD-Extract' + createDate + '.xlsx'):
#     os.remove('HC_DPD-Extract' + createDate + '.xlsx')

# writer = pd.ExcelWriter('HC_DPD-Extract_' + createDate + '.xlsx')
# final_dataset.to_excel(writer, 'ParisExtract_' + createDate)
# writer.save()

## Clean Up

In [76]:
file_list = ('drug.txt', 'form.txt', 'ingred.txt', 'route.txt', 'ther.txt', 'package.txt',
    'drug_ap.txt', 'form_ap.txt', 'ingred_ap.txt', 'route_ap.txt', 'ther_ap.txt', 'package_ap.txt',
    'drug_ia.txt', 'form_ia.txt', 'ingred_ia.txt', 'route_ia.txt', 'ther_ia.txt', 'package_ia.txt',
    'drug_dr.txt', 'form_dr.txt', 'ingred_dr.txt', 'route_dr.txt', 'ther_dr.txt', 'package_dr.txt', 'inactive.txt')

#archive_list = ('allfiles.zip', 'allfiles_ia.zip', 'allfiles_ap.zip', 'allfiles_dr.zip')

for file in file_list:
    if os.path.exists(file):
        os.remove(file)
    
try:
    archive.close()
    archive_ap.close()
    archive_dr.close()
    archive_ia.close()
except NameError as e:
    print('archive not open.  {}'.format(e))

try:
    archive.close()
    archive_ap.close()
    archive_dr.close()
    archive_ia.close()
    for arc in archive_list:
        if os.path.exists(arc):
            os.remove(arc)
except PermissionError as e:
    print ('Archive in use. {}'.format(e))

Archive in use. [WinError 32] The process cannot access the file because it is being used by another process: 'allfiles.zip'


## Create Email and attach Extract and send to Recipient

In [25]:
# import win32com.client as win32
# outlook = win32.Dispatch('outlook.application')
# mail = outlook.CreateItem(0)
# #mail.To = ''
# mail.CC = ''
# mail.Subject = ''
# mail.Body = 'kljadkjhfkdjshaf'

# mail.Attachments.Add('')
# mail.send

True

## Move Extract to Paris DB Location

In [26]:
# import os
# import shutil

# #src_dir = '.'
# #dst_dir = ''
# #src_file = ''
# #dst_file = dst_dir
# #if os.path.exists(dst_file):
# #    os.remove(dst_file)
# #    shutil.move(src_file, dst_dir)